### 핵심 동작 원리

첫 번째 카드: 항상 랜덤 선택
기억 검색: 첫 번째 카드와 같은 색상을 기억에서 찾기
두 번째 카드 선택:

기억에 있으면 → 그 위치로 직접 이동 <br>
기억에 없으면 → 랜덤 선택


메모리 관리: FIFO 방식으로 오래된 것부터 제거
매칭 성공시: 해당 카드들을 기억에서 제거

## 현재 게임 규칙

1단계2x2 / 총 그림 2개- 기본 카드: 2개- 트릭 카드: 0개제한 시간: 5초성공 리워드: 2원실패 리워드: 1원

2단계2x3 / 총 그림 3개- 기본 카드: 3개- 트릭 카드: 0개제한 시간: 7초성공 리워드: 2원

3단계3x4 / 총 그림 6개- 기본 카드: 6개- 트릭 카드: 0개제한 시간: 9초성공 리워드: 2원

4단계4x4 / 총 그림 8개- 기본 카드: 8개- 트릭 카드: 0개제한 시간: 13초성공 리워드: 2원

5단계4x4 / 총 그림 8개- 기본 카드: 6개- 트릭 카드: 2개제한 시간: 15초성공 리워드: 3원

6단계4x5 / 총 그림 10개- 기본 카드: 10개- 트릭 카드: 0개제한 시간: 21초성공 리워드: 3원

7단계4x5 / 총 그림 10개- 기본 카드: 8개- 트릭 카드: 2개제한 시간: 25초성공 리워드: 6원

- 재참여 : 최대 3번까지 플레이 가능. 이 경우, 1단계부터 재시작하며 광고를 시청한다. 매출: 1원
- 재시도 : 무재한 재시도 가능. 이 경우, 기존 실패한 단계에서 재시작하며 광고를 시청 한다. 매출 : 1원

In [20]:
import random
import numpy as np
from collections import deque, Counter
from typing import List, Tuple, Dict
import pandas as pd

class Card:
    """카드 클래스"""
    def __init__(self, color: int, position: int):
        self.color = color
        self.position = position
        self.is_matched = False

class LimitedMemoryPlayer:
    """최근 N개 카드만 기억하는 플레이어 클래스"""
    def __init__(self, memory_size: int = 3):
        self.memory_size = memory_size
        self.memory = {}  # position -> color 매핑
        self.memory_order = deque(maxlen=memory_size)  # FIFO 순서 관리
        
    def remember_card(self, position: int, color: int):
        """카드를 기억에 추가"""
        if position in self.memory:
            self.memory_order.remove(position)
            self.memory_order.append(position)
        else:
            if len(self.memory) >= self.memory_size:
                oldest_pos = self.memory_order.popleft()
                del self.memory[oldest_pos]
            
            self.memory[position] = color
            self.memory_order.append(position)
    
    def find_matching_card(self, target_color: int, exclude_positions: List[int] = None) -> int:
        """기억에서 같은 색상의 카드 위치 찾기"""
        if exclude_positions is None:
            exclude_positions = []
            
        for position, color in self.memory.items():
            if color == target_color and position not in exclude_positions:
                return position
        return -1
    
    def remove_from_memory(self, positions: List[int]):
        """매칭된 카드들을 기억에서 제거"""
        for pos in positions:
            if pos in self.memory:
                del self.memory[pos]
                if pos in self.memory_order:
                    self.memory_order.remove(pos)

def create_deck(n_pairs: int) -> List[Card]:
    """n개 쌍의 카드 덱 생성"""
    deck = []
    colors = list(range(n_pairs)) * 2
    random.shuffle(colors)
    
    for i, color in enumerate(colors):
        deck.append(Card(color, i))
    
    return deck

def get_available_positions(deck: List[Card]) -> List[int]:
    """아직 매칭되지 않은 카드 위치들 반환"""
    return [i for i, card in enumerate(deck) if not card.is_matched]

def simulate_single_game(n_pairs: int, memory_size: int) -> Tuple[int, bool]:
    """단일 게임 시뮬레이션"""
    deck = create_deck(n_pairs)
    player = LimitedMemoryPlayer(memory_size)
    turns = 0
    matched_pairs = 0
    
    while matched_pairs < n_pairs:
        turns += 1
        
        # 사용 가능한 카드들
        available_positions = get_available_positions(deck)
        
        if len(available_positions) < 2:
            break
        
        # 첫 번째 카드 선택 (랜덤)
        first_pos = random.choice(available_positions)
        first_card = deck[first_pos]
        
        # 첫 번째 카드를 기억에 추가
        player.remember_card(first_pos, first_card.color)
        
        # 기억에서 같은 색상의 카드 찾기
        matching_pos = player.find_matching_card(first_card.color, [first_pos])
        
        if matching_pos != -1 and matching_pos in available_positions:
            # 기억에서 매칭 카드를 찾은 경우
            second_pos = matching_pos
            second_card = deck[second_pos]
            
            # 매칭 성공
            first_card.is_matched = True
            second_card.is_matched = True
            matched_pairs += 1
            
            # 매칭된 카드들을 기억에서 제거
            player.remove_from_memory([first_pos, second_pos])
        else:
            # 기억에 없으므로 랜덤하게 두 번째 카드 선택
            available_for_second = [pos for pos in available_positions if pos != first_pos]
            
            if not available_for_second:
                break
                
            second_pos = random.choice(available_for_second)
            second_card = deck[second_pos]
            
            # 두 번째 카드를 기억에 추가
            player.remember_card(second_pos, second_card.color)
            
            if first_card.color == second_card.color:
                # 운 좋게 매칭
                first_card.is_matched = True
                second_card.is_matched = True
                matched_pairs += 1
                
                # 매칭된 카드들을 기억에서 제거
                player.remove_from_memory([first_pos, second_pos])
    
    return turns, matched_pairs == n_pairs

def calculate_turn_probability_density(turns_list: List[int], max_turns: int) -> Dict[int, float]:
    """각 턴당 확률 밀도 계산"""
    turn_counter = Counter(turns_list)
    total_games = len(turns_list)
    
    probability_density = {}
    for turn in range(1, max_turns + 1):
        count = turn_counter.get(turn, 0)
        probability_density[f"turn_{turn}"] = count / total_games
    
    return probability_density

def run_detailed_simulation(n_pairs: int, memory_size: int, num_simulations: int = 10000) -> Dict:
    """상세 결과를 포함한 시뮬레이션 실행"""
    turns_results = []
    successful_games = 0
    
    # 시뮬레이션 실행
    for _ in range(num_simulations):
        turns, success = simulate_single_game(n_pairs, memory_size)
        turns_results.append(turns)
        if success:
            successful_games += 1
    
    # 기본 통계 계산
    min_turns = min(turns_results)
    max_turns = max(turns_results)
    average_turns = np.mean(turns_results)
    std_turns = np.std(turns_results)
    success_rate = successful_games / num_simulations
    
    # 확률 밀도 계산
    probability_density = calculate_turn_probability_density(turns_results, max_turns)
    
    # 결과 딕셔너리 구성
    result = {
        'n_pairs': n_pairs,
        'memory_size': memory_size,
        'num_simulations': num_simulations,
        'min_turns': min_turns,
        'max_turns': max_turns,
        'average_turns': round(average_turns, 3),
        'std_turns': round(std_turns, 3),
        'success_rate': round(success_rate, 4),
        **probability_density  # turn_1, turn_2, ... 확률 밀도 추가
    }
    
    return result

def run_batch_detailed_simulation(n_pairs_list: List[int], 
                                memory_sizes: List[int], 
                                num_simulations: int = 10000) -> List[Dict]:
    """여러 조건에 대한 배치 시뮬레이션"""
    results = []
    
    for n_pairs in n_pairs_list:
        for memory_size in memory_sizes:
            print(f"시뮬레이션 중: {n_pairs}쌍, 메모리 크기 {memory_size}...")
            result = run_detailed_simulation(n_pairs, memory_size, num_simulations)
            results.append(result)
    
    return results

def normalize_probability_densities(results: List[Dict]) -> List[Dict]:
    """확률 밀도를 가장 큰 max_turns에 맞춰 정규화"""
    # 모든 결과에서 최대 턴 수 찾기
    global_max_turns = max(result['max_turns'] for result in results)
    
    normalized_results = []
    for result in results:
        normalized_result = result.copy()
        
        # 부족한 턴에 대해 0.0으로 채우기
        for turn in range(1, global_max_turns + 1):
            turn_key = f"turn_{turn}"
            if turn_key not in normalized_result:
                normalized_result[turn_key] = 0.0
        
        normalized_results.append(normalized_result)
    
    return normalized_results

def results_to_dataframe(results: List[Dict]) -> pd.DataFrame:
    """결과를 DataFrame으로 변환"""
    # 정규화된 결과 생성
    normalized_results = normalize_probability_densities(results)
    
    # DataFrame 생성
    df = pd.DataFrame(normalized_results)
    
    # 열 순서 정렬 (기본 정보 + turn_1, turn_2, ...)
    basic_cols = ['n_pairs', 'memory_size', 'num_simulations', 'min_turns', 
                  'max_turns', 'average_turns', 'std_turns', 'success_rate']
    
    turn_cols = [col for col in df.columns if col.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))  # turn_1, turn_2, ... 순서로 정렬
    
    column_order = basic_cols + turn_cols
    df = df[column_order]
    
    return df

def print_summary_statistics(results: List[Dict]):
    """요약 통계 출력"""
    print("\n=== 시뮬레이션 결과 요약 ===")
    print("=" * 80)
    
    for result in results:
        n_pairs = result['n_pairs']
        memory_size = result['memory_size']
        avg_turns = result['average_turns']
        std_turns = result['std_turns']
        min_turns = result['min_turns']
        max_turns = result['max_turns']
        success_rate = result['success_rate']
        
        print(f"\n【{n_pairs}쌍, 메모리 {memory_size}개】")
        print(f"  평균 턴 수: {avg_turns:.3f} ± {std_turns:.3f}")
        print(f"  범위: {min_turns} ~ {max_turns} 턴")
        print(f"  성공률: {success_rate:.1%}")
        
        # 확률이 높은 상위 3개 턴 출력
        turn_probs = [(k, v) for k, v in result.items() if k.startswith('turn_') and v > 0]
        turn_probs.sort(key=lambda x: x[1], reverse=True)
        
        print("  가장 빈번한 턴 수:")
        for turn_key, prob in turn_probs[:3]:
            turn_num = turn_key.split('_')[1]
            print(f"    {turn_num}턴: {prob:.1%}")
def extract_probability_density_dataframe(results: List[Dict]) -> pd.DataFrame:
    """인덱스: (n_pairs, memory_size), 컬럼: turn_1 ~ turn_n 형태의 DataFrame 추출"""
    # 정규화된 결과 생성
    normalized_results = normalize_probability_densities(results)
    
    # DataFrame 생성
    df = pd.DataFrame(normalized_results)
    
    # turn 컬럼들만 추출 및 정렬
    turn_cols = [col for col in df.columns if col.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))
    
    # 필요한 컬럼들만 선택
    probability_df = df[['n_pairs', 'memory_size'] + turn_cols].copy()
    
    # 인덱스 설정
    probability_df.set_index(['n_pairs', 'memory_size'], inplace=True)
    
    return probability_df
# 실행 예제
if __name__ == "__main__":
    print("=== 상세 결과 카드 게임 시뮬레이션 ===\n")
    
    # 시뮬레이션 설정
    n_pairs_list = [2, 3, 4]  # 카드 쌍 수
    memory_sizes = [1, 2, 3, 5]  # 메모리 크기
    num_simulations = 5000  # 시뮬레이션 횟수
    
    # 배치 시뮬레이션 실행
    results = run_batch_detailed_simulation(n_pairs_list, memory_sizes, num_simulations)
    
    # 요약 통계 출력
    print_summary_statistics(results)
    
    # DataFrame으로 변환
    df = results_to_dataframe(results)
    
    print(f"\n=== DataFrame 형태 결과 ===")
    print("형태:", df.shape)
    print("\n기본 정보 열:")
    basic_cols = ['n_pairs', 'memory_size', 'average_turns', 'min_turns', 'max_turns', 'success_rate']
    print(df[basic_cols].to_string(index=False))
    
    print(f"\n=== 확률 밀도 예시 (첫 번째 결과) ===")
    first_result = results[0]
    print(f"{first_result['n_pairs']}쌍, 메모리 {first_result['memory_size']}개:")
    
    turn_cols = [k for k in first_result.keys() if k.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))
    
    for turn_key in turn_cols[:10]:  # 처음 10턴만 출력
        turn_num = turn_key.split('_')[1]
        prob = first_result[turn_key]
        if prob > 0:
            print(f"  {turn_num}턴: {prob:.4f} ({prob:.1%})")
    
    # 이론적 완벽 플레이어와 비교
    print(f"\n=== 이론적 완벽 플레이어 비교 ===")
    for n_pairs in n_pairs_list:
        theoretical_perfect = 1.61 * n_pairs
        print(f"\n{n_pairs}쌍:")
        print(f"  이론적 완벽 플레이어: {theoretical_perfect:.2f}턴")
        
        # 해당 쌍에 대한 결과들 출력
        pair_results = [r for r in results if r['n_pairs'] == n_pairs]
        for result in pair_results:
            memory_size = result['memory_size']
            avg_turns = result['average_turns']
            diff = avg_turns - theoretical_perfect
            print(f"  메모리 {memory_size}개: {avg_turns:.2f}턴 (차이: {diff:+.2f})")
    
    # 전체 결과 반환을 위한 함수도 제공
    def get_simulation_results():
        """시뮬레이션 결과 반환 (다른 스크립트에서 사용 가능)"""
        return results, df, prob_df

=== 상세 결과 카드 게임 시뮬레이션 ===

시뮬레이션 중: 2쌍, 메모리 크기 1...
시뮬레이션 중: 2쌍, 메모리 크기 2...
시뮬레이션 중: 2쌍, 메모리 크기 3...
시뮬레이션 중: 2쌍, 메모리 크기 5...
시뮬레이션 중: 3쌍, 메모리 크기 1...
시뮬레이션 중: 3쌍, 메모리 크기 2...
시뮬레이션 중: 3쌍, 메모리 크기 3...
시뮬레이션 중: 3쌍, 메모리 크기 5...
시뮬레이션 중: 4쌍, 메모리 크기 1...
시뮬레이션 중: 4쌍, 메모리 크기 2...
시뮬레이션 중: 4쌍, 메모리 크기 3...
시뮬레이션 중: 4쌍, 메모리 크기 5...

=== 시뮬레이션 결과 요약 ===

【2쌍, 메모리 1개】
  평균 턴 수: 4.012 ± 2.395
  범위: 2 ~ 26 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    2턴: 32.3%
    3턴: 21.9%
    4턴: 15.7%

【2쌍, 메모리 2개】
  평균 턴 수: 3.332 ± 1.498
  범위: 2 ~ 17 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    2턴: 33.6%
    3턴: 33.5%
    4턴: 15.8%

【2쌍, 메모리 3개】
  평균 턴 수: 2.956 ± 0.894
  범위: 2 ~ 8 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    3턴: 43.9%
    2턴: 33.8%
    4턴: 16.7%

【2쌍, 메모리 5개】
  평균 턴 수: 2.941 ± 0.889
  범위: 2 ~ 10 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    3턴: 44.6%
    2턴: 34.0%
    4턴: 16.3%

【3쌍, 메모리 1개】
  평균 턴 수: 8.991 ± 5.080
  범위: 3 ~ 43 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    6턴: 11.3%
    5턴: 10.2%
    4턴: 9.7%

【3쌍, 메모리 2개】
  평균 턴 수: 6.742 ± 2.932
 

In [ ]:
results = run_batch_detailed_simulation([2, 3, 6,8,10], [1, 2, 3, 4,5,6,7,8,9,10,11,12], 20000)

# 요청하신 형태의 DataFrame 추출
prob_df = extract_probability_density_dataframe(results)
prob_df

In [17]:
import pandas as pd
import numpy as np
import random
from scipy import stats
from typing import Dict, List, Tuple
from collections import defaultdict

class GameLevel:
    """게임 레벨 정보"""
    def __init__(self, level: int, n_pairs: int, time_limit: float, 
                 success_reward: int, fail_reward: int = 1, 
                 turn_duration: float = 0.88, trick_cards: int = 0):
        self.level = level
        self.n_pairs = n_pairs
        self.time_limit = time_limit
        self.success_reward = success_reward
        self.fail_reward = fail_reward
        self.turn_duration = turn_duration
        self.trick_cards = trick_cards  # 트릭카드 개수
        self.basic_cards = n_pairs - trick_cards  # 기본카드 개수
        
        # 트릭카드로 인한 체감 난이도 증가를 turn_duration에 반영
        if trick_cards > 0:
            # 트릭카드 1개당 20% 속도 감소 (더 신중하게 플레이)
            self.effective_turn_duration = turn_duration * (1 + trick_cards * 0.1)
        elif level < 4:
            self.effective_turn_duration = turn_duration * (0.6)
        else:
            self.effective_turn_duration = turn_duration

class GameSimulator:
    """완전한 게임 시뮬레이션 엔진"""
    
    def __init__(self, 
                 probability_df: pd.DataFrame,
                 game_levels: Dict = None,
                 user_memory_distribution: Dict = None,
                 max_retries: int = 3,
                 retry_probability: float = 0.5,
                 turn_time_params: Dict = None):
        """
        초기화
        
        Args:
            probability_df: extract_probability_density_dataframe()으로 생성된 확률 분포 DataFrame
            game_levels: 게임 레벨 설정 (None이면 기본값 사용)
            user_memory_distribution: 유저 메모리 분포 (None이면 기본값 사용)
            max_retries: 최대 재시도 횟수
            retry_probability: 재시도 확률
            turn_time_params: 턴 시간 파라미터
        """
        self.prob_df = probability_df
        self.game_levels = game_levels or self._get_default_game_levels()
        self.user_memory_distribution = user_memory_distribution or self._get_default_memory_distribution()
        self.max_retries = max_retries
        self.retry_probability = retry_probability
        self.turn_time_params = turn_time_params or self._get_default_turn_time_params()
    
    def _get_default_game_levels(self) -> Dict:
        """기본 게임 레벨 설정"""
        return {
            1: GameLevel(1, 2, 5.0, 2, 1, 0.8, 0),    # 트릭카드 없음
            2: GameLevel(2, 3, 7.0, 2, 1, 0.9, 0),    # 트릭카드 없음
            3: GameLevel(3, 6, 9.0, 2, 1, 1.0, 0),    # 트릭카드 없음
            4: GameLevel(4, 8, 13.0, 2, 1, 1.1, 0),   # 트릭카드 없음
            5: GameLevel(5, 8, 15.0, 3, 1, 1.2, 2),   # 트릭카드 2개 → 1.2 * 1.4 = 1.68초
            6: GameLevel(6, 10, 21.0, 3, 1, 1.3, 0),  # 트릭카드 없음
            7: GameLevel(7, 10, 25.0, 6, 1, 1.4, 2)   # 트릭카드 2개 → 1.4 * 1.4 = 1.96초
        }
    
    def _get_default_memory_distribution(self) -> Dict:
        """기본 유저 메모리 분포"""
        return {
            1: 0.02,
            2: 0.05,
            3: 0.08,
            4: 0.15,
            5: 0.245,   # 중앙(최대)
            6: 0.19,
            7: 0.13,
            8: 0.075,
            9: 0.04,
            10: 0.02
        }

    
    def _get_default_turn_time_params(self) -> Dict:
        """기본 턴 시간 파라미터"""
        return {
            'distribution': 't',
            'df': 10,
            'loc': 0.88,
            'scale': 1,
            'min_time': 0.1
        }
    
    def sample_user_memory(self) -> int:
        """유저 메모리 크기 샘플링"""
        memory_sizes = list(self.user_memory_distribution.keys())
        probabilities = list(self.user_memory_distribution.values())
        return np.random.choice(memory_sizes, p=probabilities)
    
    def sample_turn_count(self, n_pairs: int, memory_size: int) -> int:
        """확률 분포표에서 턴 수 샘플링"""
        try:
            # MultiIndex 확인
            if isinstance(self.prob_df.index, pd.MultiIndex):
                # MultiIndex인 경우 (n_pairs, memory_size)
                available_memories = self.prob_df.index.get_level_values(1).unique()
                if memory_size not in available_memories:
                    memory_size = min(available_memories, key=lambda x: abs(x - memory_size))
                
                prob_row = self.prob_df.loc[(n_pairs, memory_size)]
            else:
                # 단일 인덱스인 경우 - memory_size만 사용
                if memory_size not in self.prob_df.index:
                    available_memories = self.prob_df.index.unique()
                    memory_size = min(available_memories, key=lambda x: abs(x - memory_size))
                
                prob_row = self.prob_df.loc[memory_size]
            
            # 0이 아닌 확률들만 추출
            non_zero_probs = prob_row[prob_row > 0]
            
            if len(non_zero_probs) == 0:
                # 확률이 모두 0인 경우 기본값 반환
                return n_pairs
            
            # 확률 정규화
            probabilities = non_zero_probs.values
            probabilities = probabilities / np.sum(probabilities)
            
            # 턴 번호 추출 (컬럼명에서 숫자 부분만 추출)
            turn_numbers = []
            for col in non_zero_probs.index:
                if str(col).startswith('turn_'):
                    turn_numbers.append(int(col.split('_')[1]))
                else:
                    turn_numbers.append(int(col))
            
            # 샘플링
            sampled_turn = np.random.choice(turn_numbers, p=probabilities)
            return sampled_turn
            
        except (KeyError, IndexError) as e:
            print(f"경고: ({n_pairs}쌍, 메모리{memory_size}) 조건이 없어서 기본값 {n_pairs} 사용. 에러: {e}")
            return n_pairs
    
    def sample_turn_time(self, turn_count: int, game_level: GameLevel = None) -> float:
        """턴당 소요 시간 샘플링 (트릭카드 영향 반영)"""
        params = self.turn_time_params
        
        # 기본 턴 지속시간
        base_turn_duration = params['loc']
        
        # 트릭카드로 인한 체감 난이도 반영
        if game_level and hasattr(game_level, 'trick_cards') and game_level.trick_cards > 0:
            # 트릭카드 1개당 20% 속도 감소 (더 신중하게 플레이)
            difficulty_multiplier = 1 + (game_level.trick_cards * 0.2)
            effective_turn_duration = base_turn_duration * difficulty_multiplier
        else:
            effective_turn_duration = base_turn_duration
        
        # 분포에 따른 샘플링
        if params['distribution'] == 't':
            # T분포 샘플링
            turn_times = stats.t.rvs(
                df=params['df'], 
                loc=effective_turn_duration,
                scale=params['scale'], 
                size=turn_count
            )
        elif params['distribution'] == 'normal':
            # 정규분포 샘플링
            turn_times = np.random.normal(
                loc=effective_turn_duration,
                scale=params['scale'], 
                size=turn_count
            )
        elif params['distribution'] == 'uniform':
            # 균등분포 샘플링
            turn_times = np.random.uniform(
                low=effective_turn_duration - params['scale'],
                high=effective_turn_duration + params['scale'],
                size=turn_count
            )
        else:
            # 기본값: 고정 시간
            turn_times = np.full(turn_count, effective_turn_duration)
        
        # 음수 시간 제거 (최소 시간 적용)
        turn_times = np.maximum(turn_times, params['min_time'])
        
        return np.sum(turn_times)
    
    def should_retry(self) -> bool:
        """재시도 여부 결정"""
        return np.random.uniform(0, 1) < self.retry_probability
    
    def simulate_level(self, level: int, memory_size: int) -> Dict:
        """단일 레벨 시뮬레이션"""
        game_level = self.game_levels[level]
        attempts = 0
        success = False
        total_cost = 0      # 우리가 유저에게 지출한 게임 리워드 비용 (성공 시 지급)
        total_revenue = 0   # 유저가 실패하여 광고 시청 매출 (실패 시 수익)
        
        for attempt in range(self.max_retries + 1):  # 최초 시도 + 재시도
            attempts += 1
            
            # 턴 수 샘플링
            turn_count = self.sample_turn_count(game_level.n_pairs, memory_size)
            
            # 소요 시간 샘플링 (트릭카드 영향 반영)
            total_time = self.sample_turn_time(turn_count, game_level)
            
            # 성공/실패 판정
            if total_time <= game_level.time_limit:
                # 성공 - 유저에게 리워드 지급 (우리 비용)
                success = True
                total_cost = game_level.success_reward
                break
            else:
                # 실패 - 광고 시청으로 수익 발생 (우리 수익)
                total_revenue += game_level.fail_reward
                
                # 마지막 시도가 아니고 재시도 의사가 있는 경우에만 계속
                if attempt < self.max_retries and self.should_retry():
                    continue
                else:
                    break
        
        return {
            'level': level,
            'memory_size': memory_size,
            'attempts': attempts,
            'success': success,
            'total_cost': total_cost,
            'total_revenue': total_revenue,
            'net_profit': total_revenue - total_cost,
            'n_pairs': game_level.n_pairs,
            'time_limit': game_level.time_limit,
            'turn_duration': game_level.turn_duration,
            'trick_cards': game_level.trick_cards
        }
    
    def simulate_full_game(self, memory_size: int) -> Dict:
        """전체 게임 시뮬레이션"""
        results = []
        current_level = 1
        total_stats = {
            'total_attempts': 0,
            'total_successes': 0,
            'total_failures': 0,
            'total_cost': 0,
            'total_revenue': 0,
            'levels_completed': 0,
            'final_level': 0
        }
        
        while current_level <= len(self.game_levels):
            level_result = self.simulate_level(current_level, memory_size)
            results.append(level_result)
            
            total_stats['total_attempts'] += level_result['attempts']
            total_stats['total_cost'] += level_result['total_cost']
            total_stats['total_revenue'] += level_result['total_revenue']
            total_stats['final_level'] = current_level
            
            if level_result['success']:
                total_stats['total_successes'] += 1
                total_stats['levels_completed'] += 1
                current_level += 1
            else:
                total_stats['total_failures'] += 1
                break
        
        total_stats['net_profit'] = total_stats['total_revenue'] - total_stats['total_cost']
        
        return {
            'memory_size': memory_size,
            'level_results': results,
            'total_stats': total_stats
        }
    
    def run_simulation(self, num_users: int = 10000) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """대규모 시뮬레이션 실행"""
        print(f"=== {num_users}명 유저 게임 시뮬레이션 시작 ===")
        
        user_results = []
        level_stats = defaultdict(lambda: {
            'attempts': 0, 'successes': 0, 'failures': 0,
            'total_cost': 0, 'total_revenue': 0
        })
        
        for user_id in range(num_users):
            if (user_id + 1) % 1000 == 0:
                print(f"진행률: {user_id + 1}/{num_users} ({(user_id + 1)/num_users*100:.1f}%)")
            
            memory_size = self.sample_user_memory()
            game_result = self.simulate_full_game(memory_size)
            user_results.append(game_result)
            
            for level_result in game_result['level_results']:
                level = level_result['level']
                level_stats[level]['attempts'] += level_result['attempts']
                level_stats[level]['total_cost'] += level_result['total_cost']
                level_stats[level]['total_revenue'] += level_result['total_revenue']
                
                if level_result['success']:
                    level_stats[level]['successes'] += 1
                else:
                    level_stats[level]['failures'] += 1
        
        user_df = self._create_user_dataframe(user_results)
        level_df = self._create_level_dataframe(level_stats)
        
        return user_df, level_df
    
    def _create_user_dataframe(self, user_results: List[Dict]) -> pd.DataFrame:
        """유저 결과 DataFrame 생성"""
        rows = []
        for i, result in enumerate(user_results):
            row = {
                'user_id': i + 1,
                'memory_size': result['memory_size'],
                **result['total_stats']
            }
            rows.append(row)
        
        return pd.DataFrame(rows)
    
    def _create_level_dataframe(self, level_stats: Dict) -> pd.DataFrame:
        """레벨별 통계 DataFrame 생성"""
        rows = []
        for level in range(1, len(self.game_levels) + 1):
            stats = level_stats[level]
            game_level = self.game_levels[level]
            
            total_plays = stats['successes'] + stats['failures']
            success_rate = stats['successes'] / total_plays if total_plays > 0 else 0
            avg_attempts = stats['attempts'] / total_plays if total_plays > 0 else 0
            net_profit = stats['total_revenue'] - stats['total_cost']
            
            row = {
                'level': level,
                'n_pairs': game_level.n_pairs,
                'time_limit': game_level.time_limit,
                'success_reward': game_level.success_reward,
                'fail_reward': game_level.fail_reward,
                'turn_duration': game_level.turn_duration,
                'trick_cards': game_level.trick_cards,
                'total_plays': total_plays,
                'attempts': stats['attempts'],
                'successes': stats['successes'],
                'failures': stats['failures'],
                'success_rate': success_rate,
                'avg_attempts_per_play': avg_attempts,
                'total_cost': stats['total_cost'],
                'total_revenue': stats['total_revenue'],
                'net_profit': net_profit,
                'profit_per_play': net_profit / total_plays if total_plays > 0 else 0
            }
            rows.append(row)
        
        return pd.DataFrame(rows)

def print_simulation_summary(user_df: pd.DataFrame, level_df: pd.DataFrame):
    """시뮬레이션 결과 요약 출력"""
    print("\n" + "="*80)
    print("🎮 게임 시뮬레이션 결과 요약")
    print("="*80)
    
    total_users = len(user_df)
    total_revenue = user_df['total_revenue'].sum()
    total_cost = user_df['total_cost'].sum()
    net_profit = total_revenue - total_cost
    avg_levels = user_df['levels_completed'].mean()
    
    print(f"👥 전체 사용자: {total_users:,}명")
    print(f"💰 총 수익: {total_revenue:,.0f}원")
    print(f" 총 비용: {total_cost:,.0f}원")
    print(f"📊 순이익: {net_profit:,.0f}원")
    print(f"🎯 평균 완료 레벨: {avg_levels:.1f}단계")
    
    print(f"\n📈 레벨별 통계:")
    level_summary = level_df[['level', 'n_pairs', 'trick_cards', 'time_limit', 'successes', 'failures', 
                             'success_rate', 'total_cost', 'total_revenue', 'net_profit']].round(2)
    print(level_summary.to_string(index=False))
    
    print(f"\n🧠 메모리 크기별 성과:")
    memory_stats = user_df.groupby('memory_size').agg({
        'levels_completed': 'mean',
        'total_revenue': 'mean',
        'total_cost': 'mean',
        'net_profit': 'mean'
    }).round(2)
    
    print(memory_stats)

In [18]:
# 1. memory_v2에서 생성된 prob_df 사용 (이미 생성되어 있다고 가정)
# prob_df = extract_probability_density_dataframe(results)

# 2. 게임 시뮬레이터 생성
simulator = GameSimulator(prob_df)

# 3. 시뮬레이션 실행
user_df, level_df = simulator.run_simulation(num_users=10000)

# 4. 결과 출력
print_simulation_summary(user_df, level_df)

# 5. 트릭카드 영향 확인
print("\n" + "="*80)
print("트릭카드 영향 분석")
print("="*80)

# 레벨별 평균 소요 시간 확인
for level in [1, 2, 3, 4, 5, 6, 7]:
    game_level = simulator.game_levels[level]
    print(f"Level {level}: {game_level.n_pairs}쌍, 트릭카드 {game_level.trick_cards}개")
    print(f"  기본 턴 지속시간: {game_level.turn_duration:.2f}초")
    print(f"  효과적 턴 지속시간: {game_level.effective_turn_duration:.2f}초")
    print(f"  시간 증가율: {(game_level.effective_turn_duration / game_level.turn_duration - 1) * 100:.1f}%")
    print()

# 6. 결과 저장 (선택사항)
# user_df.to_csv('user_results.csv', index=False)
# level_df.to_csv('level_results.csv', index=False)

=== 10000명 유저 게임 시뮬레이션 시작 ===
진행률: 1000/10000 (10.0%)
진행률: 2000/10000 (20.0%)
진행률: 3000/10000 (30.0%)
진행률: 4000/10000 (40.0%)
진행률: 5000/10000 (50.0%)
진행률: 6000/10000 (60.0%)
진행률: 7000/10000 (70.0%)
진행률: 8000/10000 (80.0%)
진행률: 9000/10000 (90.0%)
진행률: 10000/10000 (100.0%)

🎮 게임 시뮬레이션 결과 요약
👥 전체 사용자: 10,000명
💰 총 수익: 19,320원
 총 비용: 40,233원
📊 순이익: -20,913원
🎯 평균 완료 레벨: 2.0단계

📈 레벨별 통계:
 level  n_pairs  trick_cards  time_limit  successes  failures  success_rate  total_cost  total_revenue  net_profit
     1        2            0         5.0       9237       763          0.92       18474           1542      -16932
     2        3            0         7.0       8015      1222          0.87       16030           2483      -13547
     3        6            0         9.0       2181      5834          0.27        4362          11245        6883
     4        8            0        13.0        508      1673          0.23        1016           3093        2077
     5        8            2        15.0 

In [ ]:
user_df.total_attempts.value_counts()

total_attempts
3     2929
4     2233
5     1292
6     1049
2      999
1      707
7      425
8      201
9       95
10      39
11      19
13       7
12       5
Name: count, dtype: int64

In [24]:
level_df.attempts.sum()

39366

In [25]:
level_df

,level,n_pairs,time_limit,success_reward,fail_reward,turn_duration,trick_cards,total_plays,attempts,successes,failures,success_rate,avg_attempts_per_play,total_cost,total_revenue,net_profit,profit_per_play
0,1,2,5.0,2,1,0.8,0,10000,10779,9237,763,0.923700,1.077900,18474,1542,-16932,-1.693200
1,2,3,7.0,2,1,0.9,0,9237,10498,8015,1222,0.867706,1.136516,16030,2483,-13547,-1.466602
2,3,6,9.0,2,1,1.0,0,8015,13426,2181,5834,0.272115,1.675109,4362,11245,6883,0.858765
3,4,8,13.0,2,1,1.1,0,2181,3601,508,1673,0.232921,1.651077,1016,3093,2077,0.952315
4,5,8,15.0,3,1,1.2,2,508,927,60,448,0.118110,1.824803,180,867,687,1.352362
5,6,10,21.0,3,1,1.3,0,60,81,33,27,0.550000,1.350000,99,48,-51,-0.850000
6,7,10,25.0,6,1,1.4,2,33,54,12,21,0.363636,1.636364,72,42,-30,-0.909091


In [19]:
level_df

,level,n_pairs,time_limit,success_reward,fail_reward,turn_duration,trick_cards,total_plays,attempts,successes,failures,success_rate,avg_attempts_per_play,total_cost,total_revenue,net_profit,profit_per_play
0,1,2,5.0,2,1,0.8,0,10000,10779,9237,763,0.923700,1.077900,18474,1542,-16932,-1.693200
1,2,3,7.0,2,1,0.9,0,9237,10498,8015,1222,0.867706,1.136516,16030,2483,-13547,-1.466602
2,3,6,9.0,2,1,1.0,0,8015,13426,2181,5834,0.272115,1.675109,4362,11245,6883,0.858765
3,4,8,13.0,2,1,1.1,0,2181,3601,508,1673,0.232921,1.651077,1016,3093,2077,0.952315
4,5,8,15.0,3,1,1.2,2,508,927,60,448,0.118110,1.824803,180,867,687,1.352362
5,6,10,21.0,3,1,1.3,0,60,81,33,27,0.550000,1.350000,99,48,-51,-0.850000
6,7,10,25.0,6,1,1.4,2,33,54,12,21,0.363636,1.636364,72,42,-30,-0.909091


In [27]:
user_df.total_attempts.value_counts()

total_attempts
3     2929
4     2233
5     1292
6     1049
2      999
1      707
7      425
8      201
9       95
10      39
11      19
13       7
12       5
Name: count, dtype: int64